In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install -e git+https://github.com/ncullen93/torchsample.git#egg=torchsample
!pip install visdom
!pip install nibabel
!pip install h5py  
!pip install torchsample
!pip install tensorboardX

Obtaining torchsample from git+https://github.com/ncullen93/torchsample.git#egg=torchsample
  Cloning https://github.com/ncullen93/torchsample.git to ./src/torchsample
  Running command git clone -q https://github.com/ncullen93/torchsample.git /content/src/torchsample
  Running setup.py develop for torchsample
     |████████████████████████████████| 676 kB 8.3 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=356da9e91b32d8554decec09f68f8e25985e178419700ab16f6e324f864b5a71
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5710 sha256=91e2261af72286a06cfcc6f9fe774fbb1833f782fefee22b5a00ffe2aaa5d8b9
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built visdom torchfile
     |███████████

Import the following libraries to run the model

In [ ]:
#import all libraries
import torch.optim as optim
import torch
import torch.nn as nn
from torchvision import models
import numpy as np
import os
import sys
import pickle
import torch.nn.functional as F
import torch.utils.data as data
import pandas as pd
from torch.autograd import Variable
from src.torchsample.torchsample.transforms import RandomRotate, RandomTranslate, RandomFlip, ToTensor, Compose, RandomAffine
from torchvision import transforms
from tensorboardX import SummaryWriter
import math
from sklearn import metrics
from tqdm import tqdm
from tqdm import tqdm_notebook
from torch.utils import data
from torchvision import datasets
import matplotlib.pyplot as plt
import io
import requests
from PIL import Image
from torchvision import models, transforms
import cv2
import pdb
from sklearn.linear_model import LogisticRegression

# **The first step is to set our parameters for the task at hand:**

**task** - 'acl' or 'meniscus' 

**model** - 'resnet-18' or 'alexnet'

**directory** - directory to your data 

**lr** - learning rate 

**num_epochs** - number of epochs 





In [ ]:
directory ='./gdrive/MyDrive/data/' #directory to the dataset in google drive
#model = 'resnet-18' #resnet-18 or alexnet 
task = 'acl' # The task - acl or meniscus 
lr = 1e-5 #learning rate
num_epochs = 15 # number of epochs
early_trigger = 10
early_stop = 0 

**Defining the model**

The next step is to define the model. As previously mentioned, the model can be trained using both Alexnet and resnet-18 


In [ ]:
models.resnet18(pretrained=True)

#modify the last fully connected layer to output (1) instead of (1000)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained_model =  nn.Sequential(*list(models.resnet18(pretrained=True).children())[:-1]  )    # delete the last fc layer.
        self.classifer = nn.Linear(512, 1)

    
    def forward(self, x):
        # input size of x (1, s, 3, 256, 256) where s is the number of slices in one MRI
        x = torch.squeeze(x, dim=0) #output size (s, 3, 256, 256)
        x = self.pretrained_model(x) #output size (s, 512)
        output = torch.max(x, 0, keepdim=True)[0] #output size (1, 512)
        output = self.classifer(output.squeeze(2).squeeze(2)) #output size (1)

        return output



Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

The next step is to create a dataloader. The dataloader class is utilised to load the correct classes by taking the the *directory*, *task*, *plane*, *train* and *transform* as input. 

***train*** - *True or False* depending on the task. True is used for loading data and False is used for validating data 

**transform** - a compose function for performing transformations to the images. 

In [ ]:
class Dataset(data.Dataset):
    def __init__(self, root_dir, task, plane, train=False, transform=None):
        super().__init__()
        self.task = task
        self.plane = plane
        self.root_dir = root_dir
        self.train=train
        if self.train == True:
            self.folder_path = self.root_dir + 'train/{0}/'.format(plane)
            self.records = pd.read_csv(
                self.root_dir + 'train-{0}.csv'.format(task), header=None, names=['id', 'label'])
        else:
            self.folder_path = self.root_dir + 'valid/{0}/'.format(plane)

            self.records = pd.read_csv(
                self.root_dir + 'valid-{0}.csv'.format(task), header=None, names=['id', 'label'])

        self.records['id'] = self.records['id'].map(
            lambda i: '0' * (4 - len(str(i))) + str(i))
        self.paths = [self.folder_path + filename +
                      '.npy' for filename in self.records['id'].tolist()]
        self.labels = self.records['label'].tolist()

        self.transform = transform
        
        pos = np.sum(self.labels)
        neg = len(self.labels) - pos
        self.weights = [1, neg / pos]
        

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        array = np.load(self.paths[index]) #load MRI 
        label = self.labels[index] #get label of MRI
        label = torch.FloatTensor([label]) #convert type from numpy to torch

        if self.transform: #if you are transforming it
            array = self.transform(array) #transform the image
            array = array.numpy()


        array = np.stack((array,)*3, axis=1) #the model expects dimensions of (3, 256, 256), the MRIs are greyscale of size (256, 256). Therefore, we stack the image three times to fit the dimensions for the model.
        array = torch.FloatTensor(array)

        if label.item() == 1:
            weight = np.array([self.weights[1]])
            weight = torch.FloatTensor(weight)
        else:
            weight = np.array([self.weights[0]])
            weight = torch.FloatTensor(weight)

        return array, label, weight

**TRAINING THE MODEL**

Initialise the model, optimiser, scheduler, transformation and data loader. 


In [ ]:
model = Net() #initialise the model
if torch.cuda.is_available(): #if there is a GPU available, put the model on the GPU
    model = model.cuda()

optimizer = optim.Adam(model.parameters(), lr= lr, weight_decay=0.1) #define the optimiser as Adam

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, patience=4, factor=.3, threshold=1e-4, verbose=True) #define a scheduler that decreases the learning rate if there has been no reduction in validation loss is four epochs

#define a compose function that is a series of transformations on the images. 
augmentor = Compose([ 
            transforms.Lambda(lambda x: torch.Tensor(x)), #converts from numpy to tensor
            RandomRotate(25), #rotate the image by 25 degrees
            RandomTranslate([0.11, 0.11]), #blur the edges
            RandomFlip(), #flip the image
            ])


#initialise the train and validation datasets (class we defined earlier) and then initialise a Pytorch's dataloader
train_dataset_sag = Dataset(directory, task, "sagittal",train=True, transform=augmentor)
train_dataset_axial = Dataset(directory, task, "axial",train=True, transform=augmentor)
train_dataset_coronal = Dataset(directory, task, "coronal",train=True, transform=augmentor)

valid_dataset_sag = Dataset(directory, task, "sagittal", train=False, transform = None)
valid_dataset_axial = Dataset(directory, task, "axial", train=False, transform = None)
valid_dataset_coronal = Dataset(directory, task, "coronal", train=False, transform = None)

train_loader_sag = torch.utils.data.DataLoader(train_dataset_sag, batch_size=1, shuffle=True, num_workers=2, drop_last=False)
train_loader_axial = torch.utils.data.DataLoader(train_dataset_axial, batch_size=1, shuffle=True, num_workers=2, drop_last=False)
train_loader_coronal = torch.utils.data.DataLoader(train_dataset_coronal, batch_size=1, shuffle=True, num_workers=2, drop_last=False)

valid_loader_sag = torch.utils.data.DataLoader(valid_dataset_sag, batch_size=1, shuffle=-True, num_workers=2, drop_last=False)
valid_loader_axial = torch.utils.data.DataLoader(valid_dataset_axial, batch_size=1, shuffle=-True, num_workers=2, drop_last=False)
valid_loader_coronal = torch.utils.data.DataLoader(valid_dataset_coronal, batch_size=1, shuffle=-True, num_workers=2, drop_last=False)


## **Train model function**

# **Input parameters:**

**model** - initialised model 

**lr** - learned rate 

**train_loader** - train_loader function for specific plane 

**valid_loader** - valid loader function for specific plane 

**optimizer** - optimiser 

**task** - acl or meniscus 

**plane** - plane 

### **RETURNS**

Saves the best performing model to drive based on AUC
- Removes the previous best performing model 

In [ ]:
try:
    os.makedirs('./models/')
except:
  print('Directory already made')

def train_model(model, lr, train_loader, valid_loader, num_epochs, optimizer, task, plane): 
  best_val_auc = 0 

  for epoch in range(15):
        current_lr = lr

        y_preds = []
        y_trues = []
        losses = []
        _ = model.train()
        #loop through each MRI in the training set
        for i, (image, label, weight) in enumerate(train_loader):
            optimizer.zero_grad()

            #load all data onto the GPU
            if torch.cuda.is_available():
                image = image.cuda()
                label = label.cuda()
                weight = weight.cuda()

            label = label[0]
            weight = weight[0]

            #pass the MRI through the model
            prediction = model.forward(image.float()).squeeze(0)

            #calculate the loss
            loss = torch.nn.BCEWithLogitsLoss(weight=weight)(prediction, label)
            loss.backward() #back propagation
            optimizer.step()

            loss_value = loss.item()
            losses.append(loss_value)

            probas = torch.sigmoid(prediction) #convert output of model (logits) to a value between zero and one. This can be interpretted as a probability

            y_trues.append(int(label[0]))
            y_preds.append(probas[0].item())

            try:
                auc = metrics.roc_auc_score(y_trues, y_preds)
            except:
                auc = 0.5

            train_loss = np.round(np.mean(losses), 4)
            train_auc = np.round(auc, 4)

        #evaluate the model on the validation data after each epoch
        _ = model.eval()
        y_trues = []
        y_preds = []
        losses = []
        for i, (image, label, weight) in enumerate(valid_loader):

          if torch.cuda.is_available():
              image = image.cuda()
              label = label.cuda()
              weight = weight.cuda()

          label = label[0]
          weight = weight[0]

          prediction = model.forward(image.float()).squeeze(0)

          loss = torch.nn.BCEWithLogitsLoss(weight=weight)(prediction, label)

          loss_value = loss.item()
          losses.append(loss_value)

          probas = torch.sigmoid(prediction)

          y_trues.append(int(label[0]))
          y_preds.append(probas[0].item())

          try:
              auc = metrics.roc_auc_score(y_trues, y_preds)
          except:
              auc = 0.5

          val_loss = np.round(np.mean(losses), 4)
          val_auc = np.round(auc, 4)

        if val_auc > best_val_auc:
          best_val_auc = val_auc
          early_stop=0
          for f in os.listdir('./models/'):
                    if (task in f) and (plane in f):
                        os.remove(f'./models/{f}')
          file_name = f'model_{task}_{plane}_val_auc_{val_auc:0.4f}_train_auc_{train_auc:0.4f}_epoch_{epoch+1}.pth'
          torch.save(model, f'./models/{file_name}')
        else:
          early_stop+= 1

        if early_stop == early_trigger:
          print('Early stopping after {} epochs'.format(epoch))
          sys.exit()
        scheduler.step(val_loss)

        print("epoch : {0} | train loss : {1} | train auc {2} | val loss {3} | val auc {4} ".format(
            epoch, train_loss, train_auc, val_loss, val_auc))

        
        print('-' * 30)

In [ ]:
train_model(model, lr, train_loader_sag, valid_loader_sag, num_epochs, optimizer, task, "sagittal")
train_model(model, lr, train_loader_sag, valid_loader_sag, num_epochs, optimizer, task, "axial")
train_model(model, lr, train_loader_sag, valid_loader_sag, num_epochs, optimizer, task, "coronal")

In [ ]:
train_model(model, lr, train_loader_sag, valid_loader_sag, num_epochs, optimizer, task, "coronal")

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch : 0 | train loss : 1.1379 | train auc 0.5625 | val loss 0.7634 | val auc 0.5704 
------------------------------
epoch : 1 | train loss : 1.0686 | train auc 0.6789 | val loss 0.7567 | val auc 0.5962 
------------------------------
epoch : 2 | train loss : 0.9613 | train auc 0.7911 | val loss 0.6793 | val auc 0.7326 
------------------------------
epoch : 3 | train loss : 0.8531 | train auc 0.8329 | val loss 0.6236 | val auc 0.8272 
------------------------------
epoch : 4 | train loss : 0.7925 | train auc 0.8541 | val loss 0.5602 | val auc 0.8333 
------------------------------
epoch : 5 | train loss : 0.6975 | train auc 0.8912 | val loss 0.6201 | val auc 0.8903 
------------------------------
epoch : 6 | train loss : 0.5875 | train auc 0.926 | val loss 0.5197 | val auc 0.9127 
------------------------------
epoch : 7 | train loss : 0.5071 | train auc 0.9458 | val loss 0.8139 | val auc 0.9209 
------------------------------
epoch : 8 | train loss : 0.4858 | train auc 0.9481 | val 

# **Extract the predictions for logistic regression function**

In [ ]:
ref - https://towardsdatascience.com/mrnet-competition-part-1-65fcfb1cfa5f

def logistic_regression_prep(task, plane, train=True):
    assert task in ['acl', 'meniscus', 'abnormal']
    assert plane in ['axial', 'coronal', 'sagittal']
    
    models = os.listdir('/content/models')

    model_name = list(filter(lambda name: task in name and plane in name, models))[0]
    model_path = f'/content/models/{model_name}'

    model_trained = torch.load(model_path)
    _ = model_trained.eval()
    
    train_dataset = Dataset('./gdrive/MyDrive/data/', 
                              task, 
                              plane, 
                              train=True)
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)
    
    
    
    predictions = []
    labels = []
    with torch.no_grad():
        for image, label, _ in train_loader:
            logit = model_trained(image.cuda())
            prediction = torch.sigmoid(logit)
            predictions.append(prediction.item())
            labels.append(label.item())

    return predictions, labels

In [ ]:
results = {}

for plane in ['axial', 'coronal', 'sagittal']:
    predictions, labels = logistic_regression_prep(task, plane, train = True)
    results['labels'] = labels
    results[plane] = predictions
    
X = np.zeros((len(predictions), 3))
X[:, 0] = results['axial']
X[:, 1] = results['coronal']
X[:, 2] = results['sagittal']

y = np.array(labels)

logreg = LogisticRegression(solver='lbfgs')
logreg.fit(X, y)

In [ ]:
results_val = {}

for plane in ['axial', 'coronal', 'sagittal']:
    predictions, labels = logistic_regression_prep(task, plane, train=False)
    results_val['labels'] = labels
    results_val[plane] = predictions

predictions
X_val = np.zeros((len(predictions), 3))
X_val[:, 0] = results_val['axial']
X_val[:, 1] = results_val['coronal']
X_val[:, 2] = results_val['sagittal']

y_pred = logreg.predict_proba(X_val)[:, 1]
metrics.roc_auc_score(results_val['labels'], y_pred)

**GRAD-CAM**

Select models which you would like to use to produce heatmaps from GRAD-CAMS 

model_sag = model on saggital plane

model_axial - model on axial plane 

model_coronal - model on coronal plane 

In [ ]:
model_sag = torch.load('/content/models/model_meniscus_axial_val_auc_0.6751_train_auc_0.9944_epoch_15.pth',map_location=torch.device('cpu'))
model_axial = torch.load('/content/models/model_meniscus_axial_val_auc_0.6751_train_auc_0.9944_epoch_15.pth',map_location=torch.device('cpu'))
model_coronal = torch.load('/content/models/model_meniscus_coronal_val_auc_0.6612_train_auc_0.8449_epoch_10.pth',map_location=torch.device('cpu'))

In [ ]:
# forward hook
features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

model_sag._modules.get('pretrained_model')[7].register_forward_hook(hook_feature);
model_axial._modules.get('pretrained_model')[7].register_forward_hook(hook_feature);
model_coronal._modules.get('pretrained_model')[7].register_forward_hook(hook_feature);

#backward hook
features_blobs_gr = []

def backward_hook(module, grad_in, grad_out):
    _ =features_blobs_gr.append(grad_out[0].cpu().numpy() )
    
model_sag._modules.get('pretrained_model')[7].register_backward_hook(backward_hook);
model_axial._modules.get('pretrained_model')[7].register_backward_hook(backward_hook);
model_coronal._modules.get('pretrained_model')[7].register_backward_hook(backward_hook);

_=mod.eval()

In [ ]:
train_dataset = Dataset(directory, task,'coronal',transform=None)

ind = list(range(0,1130))

def gradcam(train_dataset, ind):
  for i in range(0,len(ind)):
      image,label,weight = train_dataset.__getitem__(ind[i])
      if len(str(ind[i])) == 1:
          string = '000' + str(ind[i]) 
      elif len(str(ind[i])) == 2:
          string = '00' + str(ind[i]) 
      elif len(str(ind[i])) == 3:
          string = '0' + str(ind[i]) 
      elif len(str(ind[i])) == 4:
          string = str(i) 

      task = 'acl'
      plane ='sagittal'
      split = 'valid'

      try:
          os.makedirs('./cams7/'+ split +'/'  + plane +'/' + string)
      except:
          print('')
      try:
          os.makedirs('./heatmaps7/'+ split +'/' + plane +'/' + string)
      except:
          print('')
      try:
          os.makedirs('./img7/'+ split +'/' + plane +'/' + string)
      except:
          print('')
      try:
          os.makedirs('./cam_boundary7/'+ split +'/' + plane +'/' + string)
      except:
          print('')

      
      prediction = mod.forward(image.float())
      probas = torch.sigmoid(prediction)
      loss = torch.nn.BCEWithLogitsLoss(weight=weight)(prediction.squeeze(0), label)
      loss.backward()
      weights =nn.AdaptiveAvgPool2d(1)(torch.Tensor(features_blobs_gr[i]))
      x = torch.Tensor(features_blobs[i])
      gcam = torch.mul(x, weights).sum(dim=1, keepdim=True)
      gcam = F.relu(gcam)
      gcam = F.interpolate(gcam, (256,256), mode="bilinear", align_corners=False)
      B, C, H, W = gcam.shape


      gcam = gcam.view(B, -1)
      gcam -= gcam.min(dim=1, keepdim=True)[0]
      gcam /= gcam.max(dim=1, keepdim=True)[0]
      gcam = gcam.view(B, H, W)


      for j in range(0, image.shape[0]):
          img = image[j].numpy()
          img = img.transpose(1, 2, 0)
          heatmap = np.uint8(255 * gcam.numpy()[j])

          heatmap = (cv2
                      .cvtColor(cv2.applyColorMap(
                          cv2.resize(heatmap, (256, 256)),
                          cv2.COLORMAP_JET), 
                                cv2.COLOR_BGR2RGB)
                    )
          
          
          #Heatmap 
          heatmap1 = heatmap.copy()
          
          #Split images into respective colour channels 
          blue_img, green_img, red_img = cv2.split(heatmap)

          # Focus on red 
          red_img = cv2.bitwise_not(red_img)
          
          # Perform a threshold on the img, removing all but red (changed to black iwth bitwise_not previously)
          thresh = cv2.threshold(red_img, 254, 255, cv2.THRESH_BINARY)[1]
          cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
          cnts = cnts[0] if len(cnts) == 2 else cnts[1]
          for c in cnts:
            #Retrireve coordinates for rectange 
            x,y,w,h = cv2.boundingRect(c)
            
            # Place the bounding box on the heatmap
            cv2.rectangle(heatmap, (x, y), (x + w, y + h), (36,255,12), 2)
            #cv2.imwrite("test.png",img)

          # result - no changes 
          result =  img * 0.5

          # cam - Heatmap + MRI slice 
          cam = img * 0.5 + heatmap1 * 0.5

          # cam_boundary - Heatmap + MRI Slice + Bounding box 
          cam_boundary = img * 0.5 + heatmap * 0.5


          # Output all files to respective folders 
          pil_img_result = Image.fromarray(np.uint8(result))
          pil_img_cam = Image.fromarray(np.uint8(cam))
          pil_cam_boundary = Image.fromarray(np.uint8(cam_boundary))
          img_acc[string + "/" + str(j)] =  probas

          pil_img_cam.save('./cams7/'+split + '/' +  plane +'/' + string +'/' +str(j) + '.png')
          pil_img_result.save('./img7/'+split + '/' +  plane +'/' + string +'/' +str(j) + '.png')
          pil_cam_boundary.save('./cam_boundary7/'+split + '/' +  plane +'/' + string +'/' +str(j) + '.png')
          np.save('./heatmaps7/'+split + '/' +  plane +'/' + string +'/' +str(j) + '.npy', heatmap1)

In [ ]:
grad_cam(train_dataset_sag)
grad_cam(train_dataset_axial)
grad_cam(train_dataset_coronal)
